Notebook to make a figure that demonstrates the synthetic noise generation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from obspy.signal.filter import lowpass
import os
from pathlib import Path

import pyarrow as pa
import pyarrow.parquet as pq
import json

from scipy import signal
from obspy.signal.tf_misfit import cwt
import pywt
from scipy import stats
        
from multiprocessing import Pool, cpu_count

import sys
sys.path.insert(0, os.path.abspath('../../bin/generate_ts'))

from generate_ts_utils import *


In [ ]:
fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize-3
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

In [ ]:
class vel_ts:
    def __init__(self, array):
        self.array = array
        self.record_number= self.array[0][0]
        #self.meta_dict=make_meta_dict(self.array)

        
    def vel_load_ds(self, target_sr):
        
        path = Path(os.getcwd())
        sm_fn=glob.glob(os.path.join('../../data','ngaw2','RSN%s_*.VT2' %self.record_number))
        scale_f, dt= meta_check(sm_fn[0])
        
        # determine vertical TS
        up_idx=vertical_idx(sm_fn)

        #sort out components- only distinguish between Vert and Horiz for gnss noise
        h_idxs=np.delete(np.arange(3),up_idx)
        h0_idx=h_idxs[0]
        h1_idx=h_idxs[1]

        sr=1/dt
        deci=int(sr/target_sr)
        
        self.H0, self.time, self.H0_y=load_ds_label(sm_fn[h0_idx],sr, deci, buffer45=True)
        self.H1, self.time, self.H1_y=load_ds_label(sm_fn[h1_idx],sr, deci, buffer45=True)
        self.UP, self.time, self.UP_y=load_ds_label(sm_fn[up_idx],sr, deci, buffer45=True)


In [ ]:
vel_1=vel_ts(create_nga_event_station_list(rsn=2499)) #2499
target_sr=5
vel_1.vel_load_ds(target_sr)
name='H0'
noise_lev=50
noise_ts=create_noise_ts(n_steps=len(vel_1.H0), level=noise_lev, name=name)

In [ ]:
fig,ax=plt.subplots(3,1, sharex=True, figsize=(6,6))

trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)

ax[0].axvspan(vel_1.H0_y.nonzero()[0].min(),vel_1.H0_y.nonzero()[0].max(), color='grey', alpha=.2)
ax[1].axvspan(vel_1.H1_y.nonzero()[0].min(),vel_1.H0_y.nonzero()[0].max(), color='grey', alpha=.2)
ax[2].axvspan(vel_1.UP_y.nonzero()[0].min(),vel_1.UP_y.nonzero()[0].max(), color='grey', alpha=.2)
colors=['#7570b3','#d95f02','#1b9e77'][-3:]
levs=[95,50,5][-3:]
for i,(component,name,fig_lab) in enumerate(zip([vel_1.H0,vel_1.H1,vel_1.UP],['H0','H1','UP'],['(a)','(b)','(c)',])):
    for j,(color,noise_lev) in enumerate(zip(colors,levs)):
        noise_ts=create_noise_ts(n_steps=len(vel_1.H0), level=noise_lev, name=name)
        combined=component+noise_ts
        ax[i].plot(combined+.1*j, color=color, label=str(noise_lev)+'\%')
        ax[i].set_ylabel('%s (m/s)' %name)
        
        ax[i].set_xlim([100,500])
        
        ax[i].text(0, 1.05, fig_lab, transform=ax[i].transAxes + trans,
            fontsize='medium', verticalalignment='top')

ax[2].set_ylim([-.2,.25])
ax[0].legend(ncol=3, loc='lower left')
ax[2].set_xlabel('Time (s)')
fig.tight_layout()
plt.savefig('figs/example_ts.png',dpi=300)

# power point Fig

In [ ]:
fsize = 20
tsize = 20
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize-3
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

#plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
fig,ax=plt.subplots(3,1, sharex=True, figsize=(6,6))

ax[0].axvspan(vel_1.H0_y.nonzero()[0].min(),vel_1.H0_y.nonzero()[0].max(), color='grey', alpha=.2)
ax[1].axvspan(vel_1.H1_y.nonzero()[0].min(),vel_1.H0_y.nonzero()[0].max(), color='grey', alpha=.2)
ax[2].axvspan(vel_1.UP_y.nonzero()[0].min(),vel_1.UP_y.nonzero()[0].max(), color='grey', alpha=.2)
colors=['#7570b3','#d95f02','#1b9e77'][-3:]
colors=['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']
levs=[95,80,65,50,35,20,5]#[-3:]
for i,(component,name) in enumerate(zip([vel_1.H0,vel_1.H1,vel_1.UP],['H0','H1','UP'])):
    for j,(color,noise_lev) in enumerate(zip(colors,levs)):
        noise_ts=create_noise_ts(n_steps=len(vel_1.H0), level=noise_lev, name=name)
        combined=component+noise_ts
        ax[i].plot(combined+.1*j, color=color, label=noise_lev)
        ax[i].set_ylabel('%s (m/s)' %name)
        
        ax[i].set_xlim([250,450])

ax[2].set_ylim([-.2,.7])
ax[2].set_xticks([])#ax[0].legend(ncol=3, loc='lower left')
fig.tight_layout()
plt.savefig('example_ts_pp.png',dpi=300)

## Figure demonstrating feature selection

In [ ]:
vel_1=vel_ts(create_nga_event_station_list(rsn=4369)) #2499 #
target_sr=5
vel_1.vel_load_ds(target_sr)
name='H0'
noise_lev=50
noise_ts=create_noise_ts(n_steps=len(vel_1.H0), level=noise_lev, name=name)

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.transforms as mtransforms

per_min=.4
per_max=10
    
idx=60+150#+15*5
idx2=idx+5*30

colors=['#1b9e77','#d95f02','#7570b3','#e7298a']
ax_lim=[-.05,.08]
axx_lim=[vel_1.time[idx:idx2].min(),vel_1.time[idx:idx2].max()]
trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)

fig = plt.figure(tight_layout=True, figsize=(8,10))
gs = gridspec.GridSpec(3, 3, height_ratios=(1.5,1, 1),
                      left=0.1, right=0.85, bottom=0.15, top=0.9,
                      wspace=0.05, hspace=0.3)


ax = fig.add_subplot(gs[0, :])

ax.axvspan(vel_1.time[vel_1.H0_y>0].min(),vel_1.time[vel_1.H0_y>0].max(), color='grey', alpha=.2)
ax.plot(vel_1.time[idx:idx2],vel_1.H0[idx:idx2], label='M5.5; 12.5km Rrup', color=colors[0], lw=2)

ax.plot(vel_1.time[idx:idx2],noise_ts[idx:idx2], label='50 percentile synthetic noise', color=colors[1], lw=1)
ax.set_ylim(ax_lim)
ax.legend()
ax.set_xlim(axx_lim)
ax.text(0, 1.0, '(a)', transform=ax.transAxes + trans,
            fontsize='medium', verticalalignment='top')
ax.set_xlabel('Seconds')
ax.set_ylabel('Velocity (m/s)',labelpad=-10)

ax2 = fig.add_subplot(gs[1, :])
combined=vel_1.H0+noise_ts


abs_comb=np.abs(combined)
idx_4=abs_comb.argsort()[-4:][::-1]
ax2.scatter(vel_1.time[idx_4],combined[idx_4], color=colors[3], label='Max 4')
ax2.axhline(np.median(combined[idx:idx2]), color=colors[3], label='Median')
#ax2.axhline(stats.median_abs_deviation(combined[idx:idx2]), color=colors[3])
ax2.fill_between(vel_1.time, -stats.median_abs_deviation(combined[idx:idx2]),
                 stats.median_abs_deviation(combined[idx:idx2]),color=colors[3], alpha=0.25, label='MAD')
ax2.plot(vel_1.time[idx:idx2],combined[idx:idx2], color=colors[2], lw=3)
ax2.set_ylim(ax_lim)
ax2.set_xlim(axx_lim)
ax2.text(0, 1.0, '(b)', transform=ax2.transAxes + trans,
            fontsize='medium', verticalalignment='top')
ax2.set_ylabel('Velocity (m/s)',labelpad=-5)
ax2.legend(loc='upper right')
#ax3 = fig.add_subplot(gs[2, 0])
#ax3.plot(vel_1.time[idx:idx2],combined[idx:idx2])
#ax3.set_ylim(ax_lim)

ax4 = fig.add_subplot(gs[2, :])
f,p_sig=signal.periodogram(vel_1.H0[idx:idx2],fs=5)
f,p_noise=signal.periodogram(noise_ts[idx:idx2],fs=5)
f,p_comb=signal.periodogram(combined[idx:idx2],fs=5)
ax4.loglog(1/f, p_sig, label='Signal', color=colors[0], alpha=.5, lw=2)
ax4.loglog(1/f, p_noise, label='Noise', color=colors[1], alpha=.5, lw=1)
ax4.loglog(1/f, p_comb, label='Signal+Noise', color=colors[2], lw=3)
ax4.set_xlim([per_min,per_max*3])
ax4.legend()
ax4.text(0, 1.0, '(c)', transform=ax4.transAxes + trans,
            fontsize='medium', verticalalignment='top')

ax4.set_ylabel('PSD ')
ax4.set_xlabel('Period [s]')
#ax4.plot(1/f[0:100], p_sig[0:100], 'ro-')


#ax4.set_ylim(ax_lim)
fig.tight_layout()
plt.savefig('figs/demonstrate.png',dpi=300)
#plt.savefig('figs/feature_peformance.png',dpi=300)